In [2]:
# -*- coding: utf-8 -*-
"""
Created on Sun Jan 27 10:58:21 2019
@author: celad
"""

import numpy as np
import pandas as pd
import re
import sklearn
# import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error

In [5]:
hockey_path = 'C:\\Users\\celad\\Google Drive\\Documents\\Research\\stunning-parakeet\\hockey-data-hart.csv'
original_data = pd.read_csv(hockey_path)
original_data.describe()

,Year,age,votes,pct_of_vote,first,second,third,fourth,fifth,goals,...,plus_minus,wins_goalie,losses_goalie,ties_goalie,goals_against_avg,save_pct,ops,dps,gps,ps
count,1122.000000,1122.000000,1118.000000,1118.000000,810.000000,810.000000,737.000000,511.000000,511.000000,1122.000000,...,894.000000,241.000000,241.000000,241.000000,241.000000,209.000000,1122.000000,1122.000000,1122.000000,1122.000000
mean,1985.275401,27.868093,79.364937,12.799472,6.611111,6.614815,4.567164,4.800391,4.774951,24.928699,...,16.015660,31.398340,18.705394,8.452282,2.404108,0.916057,5.745811,1.942335,2.568360,10.257130
std,25.396093,4.363141,191.286838,20.003639,18.562842,14.447545,8.298190,7.730464,6.713335,18.674647,...,20.132923,8.414214,6.736741,3.501012,0.452803,0.012899,3.914743,1.857175,5.110044,3.129733
min,1924.000000,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-62.000000,0.000000,0.000000,0.000000,0.000000,0.875000,-0.100000,0.000000,0.000000,1.700000
25%,1970.000000,25.000000,4.000000,0.670000,0.000000,0.000000,0.000000,0.000000,1.000000,8.000000,...,0.000000,25.000000,14.000000,6.000000,2.110000,0.910000,2.500000,0.700000,0.000000,8.100000
50%,1991.000000,28.000000,18.000000,3.560000,0.000000,1.000000,1.000000,2.000000,2.000000,26.000000,...,13.000000,32.000000,19.000000,8.000000,2.340000,0.919000,6.300000,1.500000,0.000000,10.200000
75%,2007.000000,31.000000,61.000000,16.842500,2.000000,5.000000,5.000000,5.000000,6.000000,39.000000,...,26.000000,37.000000,23.000000,10.000000,2.600000,0.924000,8.600000,2.575000,0.000000,12.200000
max,2018.000000,43.000000,1604.000000,100.000000,147.000000,119.000000,63.000000,49.000000,47.000000,92.000000,...,124.000000,48.000000,47.000000,22.000000,3.980000,0.944000,17.700000,9.900000,20.900000,22.800000


In [ ]:
# encode categorial labels with value between 0 and n_classes-1
X_cat = X.select_dtypes(include=[object])
X_cat.head(3)


one_hot_encoded_training_predictors = pd.get_dummies(train_predictors)

In [ ]:
le = preprocessing.LabelEncoder()
X_2 = X_cat.apply(le.fit_transform)
X_2.head

In [17]:
# make copy to avoid changing original data (when Imputing)
new_data = original_data.copy()

In [13]:
# make new columns indicating what will be imputed
cols_with_missing = (col for col in new_data.columns 
                                 if new_data[col].isnull().any())
for col in cols_with_missing:
    new_data[col + '_was_missing'] = new_data[col].isnull()

In [18]:
# Imputation
my_imputer = SimpleImputer()

new_data = pd.DataFrame(my_imputer.fit_transform(new_data))
# new_data.columns = original_data.columns

AttributeError: 'DataFrame' object has no attribute 'dtype'

In [37]:
#split data
train_data, test_data = train_test_split(new_data)

#pick output
y_train = train.pct_of_vote
y_test = test.pct_of_vote

#combine year and player name?
features = [ 'Year',  
             'player',
             'age',
             'team_id',
             'pos',
             'goals',
             'assists',
             'points',
             'plus_minus',
             'ops',
             'dps',
             'gps',
             'ps'   ]
#input examples
X = new_data[features]

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
5        NaN
6        NaN
7        NaN
8        NaN
9        NaN
10       NaN
11       NaN
12       NaN
13       NaN
14       NaN
15       NaN
16       NaN
17       NaN
18       NaN
19       NaN
20       NaN
21       NaN
22       NaN
23       NaN
24       NaN
25       NaN
26       NaN
27       NaN
28       NaN
29       NaN
        ... 
1092     0.0
1093     0.0
1094     0.0
1095     0.0
1096     0.0
1097     0.0
1098     0.0
1099     0.0
1100     0.0
1101     0.0
1102    72.0
1103    60.0
1104    11.0
1105    10.0
1106     6.0
1107     0.0
1108     3.0
1109     1.0
1110     0.0
1111     1.0
1112     0.0
1113     0.0
1114     0.0
1115     0.0
1116     0.0
1117     0.0
1118     0.0
1119     0.0
1120     0.0
1121     0.0
Name: first, Length: 1122, dtype: float64

In [6]:
model1 = DecisionTreeRegressor(random_state=1)

In [7]:
model1.fit(X,y)

ValueError: could not convert string to float: 'G'